##### Copyright 2019 The TensorFlow Authors.

In [ ]:
#@title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# TensorFlow 2 quickstart for beginners

<table class="tfo-notebook-buttons" align="left">
  <td>
    <a target="_blank" href="https://www.tensorflow.org/tutorials/quickstart/beginner"><img src="https://www.tensorflow.org/images/tf_logo_32px.png" />View on TensorFlow.org</a>
  </td>
  <td>
    <a target="_blank" href="https://colab.research.google.com/github/tensorflow/docs/blob/master/site/en/tutorials/quickstart/beginner.ipynb"><img src="https://www.tensorflow.org/images/colab_logo_32px.png" />Run in Google Colab</a>
  </td>
  <td>
    <a target="_blank" href="https://github.com/tensorflow/docs/blob/master/site/en/tutorials/quickstart/beginner.ipynb"><img src="https://www.tensorflow.org/images/GitHub-Mark-32px.png" />View source on GitHub</a>
  </td>
  <td>
    <a href="https://storage.googleapis.com/tensorflow_docs/docs/site/en/tutorials/quickstart/beginner.ipynb"><img src="https://www.tensorflow.org/images/download_logo_32px.png" />Download notebook</a>
  </td>
</table>

This short introduction uses [Keras](https://www.tensorflow.org/guide/keras/overview) to:

1. Load a prebuilt dataset.
1. Build a neural network machine learning model that classifies images.
2. Train this neural network.
3. Evaluate the accuracy of the model.

This tutorial is a [Google Colaboratory](https://colab.research.google.com/notebooks/welcome.ipynb) notebook. Python programs are run directly in the browser—a great way to learn and use TensorFlow. To follow this tutorial, run the notebook in Google Colab by clicking the button at the top of this page.

1. In Colab, connect to a Python runtime: At the top-right of the menu bar, select *CONNECT*.
2. Run all the notebook code cells: Select *Runtime* > *Run all*.

## Set up TensorFlow

Import TensorFlow into your program to get started:

In [ ]:
import tensorflow as tf
print("TensorFlow version:", tf.__version__)

If you are following along in your own development environment, rather than [Colab](https://colab.research.google.com/github/tensorflow/docs/blob/master/site/en/tutorials/quickstart/beginner.ipynb), see the [install guide](https://www.tensorflow.org/install) for setting up TensorFlow for development.

Note: Make sure you have upgraded to the latest `pip` to install the TensorFlow 2 package if you are using your own development environment. See the [install guide](https://www.tensorflow.org/install) for details.

## Load a dataset

Load and prepare the [MNIST dataset](http://yann.lecun.com/exdb/mnist/). Convert the sample data from integers to floating-point numbers:

In [6]:
import tensorflow as tf
mnist = tf.keras.datasets.mnist

(x_train, y_train), (x_test, y_test) = mnist.load_data()
x_train, x_test = x_train / 255.0, x_test / 255.0
# from PIL import Image
# Image.fromarray(x_train[2]).show()
x_train.shape


(60000, 28, 28)

## Build a machine learning model

Build a `tf.keras.Sequential` model by stacking layers.

In [12]:
model = tf.keras.models.Sequential([
  tf.keras.layers.Flatten(input_shape=(28, 28)),
  tf.keras.layers.Dense(128, activation='relu'),
  tf.keras.layers.Dropout(0.2),
  tf.keras.layers.Dense(10)
])


[<tf.Variable 'dense_10/kernel:0' shape=(784, 128) dtype=float32, numpy=
 array([[-0.01588959, -0.07970851,  0.03854784, ...,  0.0233123 ,
         -0.07538179, -0.053054  ],
        [-0.05807555, -0.00306784,  0.03634281, ...,  0.0705281 ,
          0.01866291, -0.0228096 ],
        [-0.06139516, -0.00525414,  0.03440315, ..., -0.06173775,
          0.00531805,  0.07866786],
        ...,
        [-0.01638021,  0.00472938, -0.0546301 , ...,  0.0402629 ,
          0.07346582,  0.02595151],
        [ 0.07477041,  0.06621157,  0.07634319, ..., -0.05125552,
         -0.03676613,  0.02605459],
        [ 0.05405217, -0.05107181, -0.07108402, ..., -0.05154172,
          0.00330565, -0.02357153]], dtype=float32)>,
 <tf.Variable 'dense_10/bias:0' shape=(128,) dtype=float32, numpy=
 array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0

For each example, the model returns a vector of [logits](https://developers.google.com/machine-learning/glossary#logits) or [log-odds](https://developers.google.com/machine-learning/glossary#log-odds) scores, one for each class.

In [16]:
predictions = model(x_train[:1]).numpy()
predictions

array([[ 0.45324105,  1.075379  , -0.7473846 , -0.0397311 , -0.35279942,
        -0.3404149 ,  0.266112  ,  0.55159223, -0.36459428,  0.3794889 ]],
      dtype=float32)

The `tf.nn.softmax` function converts these logits to *probabilities* for each class: 

In [17]:
tf.nn.softmax(predictions).numpy()

array([[0.12536816, 0.23354918, 0.03773655, 0.07657591, 0.05599231,
        0.05669006, 0.10397242, 0.13832498, 0.05533576, 0.11645472]],
      dtype=float32)

Note: It is possible to bake the `tf.nn.softmax` function into the activation function for the last layer of the network. While this can make the model output more directly interpretable, this approach is discouraged as it's impossible to provide an exact and numerically stable loss calculation for all models when using a softmax output. 

Define a loss function for training using `losses.SparseCategoricalCrossentropy`, which takes a vector of logits and a `True` index and returns a scalar loss for each example.

In [19]:
loss_fn = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)

This loss is equal to the negative log probability of the true class: The loss is zero if the model is sure of the correct class.

This untrained model gives probabilities close to random (1/10 for each class), so the initial loss should be close to `-tf.math.log(1/10) ~= 2.3`.

In [20]:
loss_fn(y_train[:1], predictions).numpy()

2.8701565

Before you start training, configure and compile the model using Keras `Model.compile`. Set the [`optimizer`](https://www.tensorflow.org/api_docs/python/tf/keras/optimizers) class to `adam`, set the `loss` to the `loss_fn` function you defined earlier, and specify a metric to be evaluated for the model by setting the `metrics` parameter to `accuracy`.

In [21]:
model.compile(optimizer='adam',
              loss=loss_fn,
              metrics=['accuracy'])

## Train and evaluate your model

Use the `Model.fit` method to adjust your model parameters and minimize the loss: 

In [22]:
model.fit(x_train, y_train, epochs=5)

Epoch 1/5
1875/1875 [==============================] - 6s 3ms/step - loss: 0.2981 - accuracy: 0.9130
Epoch 2/5
1875/1875 [==============================] - 5s 3ms/step - loss: 0.1454 - accuracy: 0.9566
Epoch 3/5
1875/1875 [==============================] - 5s 3ms/step - loss: 0.1084 - accuracy: 0.9669
Epoch 4/5
1875/1875 [==============================] - 5s 3ms/step - loss: 0.0908 - accuracy: 0.9716
Epoch 5/5
1875/1875 [==============================] - 4s 2ms/step - loss: 0.0752 - accuracy: 0.9761


The `Model.evaluate` method checks the models performance, usually on a "[Validation-set](https://developers.google.com/machine-learning/glossary#validation-set)" or "[Test-set](https://developers.google.com/machine-learning/glossary#test-set)".

In [ ]:
model.evaluate(x_test,  y_test, verbose=2)

The image classifier is now trained to ~98% accuracy on this dataset. To learn more, read the [TensorFlow tutorials](https://www.tensorflow.org/tutorials/).

If you want your model to return a probability, you can wrap the trained model, and attach the softmax to it:

In [23]:
probability_model = tf.keras.Sequential([
  model,
  tf.keras.layers.Softmax()
])

In [24]:
probability_model(x_test[:5])

<tf.Tensor: shape=(5, 10), dtype=float32, numpy=
array([[8.64942677e-08, 5.87331384e-09, 1.05526815e-05, 2.70386663e-04,
        3.94399227e-12, 4.86351894e-07, 3.05013820e-14, 9.99709785e-01,
        1.92696035e-07, 8.53710571e-06],
       [1.35945896e-07, 9.68022050e-06, 9.99957561e-01, 3.22908272e-05,
        3.80731471e-13, 1.79660731e-07, 1.36386813e-08, 6.68442191e-15,
        7.94515600e-08, 3.46461446e-14],
       [3.97157510e-06, 9.97927070e-01, 2.34972133e-04, 1.55611022e-04,
        1.41515426e-04, 1.52342764e-05, 5.56932137e-05, 9.42864572e-04,
        5.16462431e-04, 6.56619250e-06],
       [9.99985576e-01, 9.13844000e-10, 1.25574034e-05, 5.07096871e-08,
        5.76898884e-09, 7.15034389e-07, 5.18823697e-07, 2.71918196e-07,
        1.68111534e-08, 2.00621926e-07],
       [7.14412238e-07, 5.61470728e-08, 1.21964567e-05, 5.91814626e-07,
        9.63649988e-01, 2.05746787e-07, 5.77079618e-07, 1.24153637e-04,
        1.40369766e-05, 3.61975618e-02]], dtype=float32)>

## Conclusion

Congratulations! You have trained a machine learning model using a prebuilt dataset using the [Keras](https://www.tensorflow.org/guide/keras/overview) API.

For more examples of using Keras, check out the [tutorials](https://www.tensorflow.org/tutorials/keras/). To learn more about building models with Keras, read the [guides](https://www.tensorflow.org/guide/keras). If you want learn more about loading and preparing data, see the tutorials on [image data loading](https://www.tensorflow.org/tutorials/load_data/images) or [CSV data loading](https://www.tensorflow.org/tutorials/load_data/csv).
